In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist

# Завантаження та підготовка даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

num_classes = 10
input_shape = (28, 28, 1)

# Створення архітектури згорткової нейромережі
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

# Навчання моделі
batch_size = 128
epochs = 15

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Оцінка точності моделі на тестових даних
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.applications import VGG16

# Завантаження та підготовка даних
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Збільшення розміру зображень до 32x32
x_train_resized = np.pad(x_train, ((0,0), (2,2), (2,2)), mode='constant', constant_values=0)
x_test_resized = np.pad(x_test, ((0,0), (2,2), (2,2)), mode='constant', constant_values=0)

# Розширення зображень для отримання трьох каналів
x_train_resized = np.expand_dims(x_train_resized, -1)
x_test_resized = np.expand_dims(x_test_resized, -1)
x_train_resized = np.repeat(x_train_resized, 3, axis=-1)
x_test_resized = np.repeat(x_test_resized, 3, axis=-1)

x_train_resized = keras.applications.vgg16.preprocess_input(x_train_resized)
x_test_resized = keras.applications.vgg16.preprocess_input(x_test_resized)

num_classes = 10

# Завантаження та ініціалізація моделі VGG16 без верхніх повнозв'язаних шарів
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Замороження ваг моделі VGG16
base_model.trainable = False

# Додавання верхніх повнозв'язаних шарів для класифікації
model = keras.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

# Навчання моделі
batch_size = 128
epochs = 10

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train_resized, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Оцінка точності моделі на тестових даних
score = model.evaluate(x_test_resized, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

58889256/58889256 [==============================] - 15s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14848586 (56.64 MB)
Trainable params: 133898 (523.04 KB)
Non-trainable params: 14714688 (56.13 MB)
_______